# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
# Import API key
from api_keys import geoapify_key

In [43]:
url = "https://api.geoapify.com/v2/places"
city = "London"
query_url = url + "appid=" + geoapify_key + "&q="
query_url

'https://api.geoapify.com/v2/placesappid=39c3c5a74e1e451990dce7050c564229&q='

In [35]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/Users/bolit/Python-api-Challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tyukhtet,56.5389,89.3011,289.46,97,100,0.93,RU,1690630461
1,1,edinburgh of the seven seas,-37.0676,-12.3116,287.81,88,97,11.61,SH,1690630461
2,2,udachny,66.4167,112.4000,299.51,25,0,2.58,RU,1690630461
3,3,okap,19.7578,-72.2042,300.50,75,98,1.19,HT,1690630461
4,4,waitangi,-43.9535,-176.5597,283.05,96,82,1.79,NZ,1690630461


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:


# Configure the map plot
# Create a map centered at a specific location (e.g., let's assume the first city in the DataFrame)
map_center = [city_data_df['Lat'].iloc[0], city_data_df['Lng'].iloc[0]]
map_zoom = 5  # Adjust the zoom level as needed

# Create the map
mymap = folium.Map(location=map_center, zoom_start=map_zoom)

# Add a point for each city with the size representing the humidity
for index, row in city_data_df.iterrows():
    popup_text = f"City: {row['City']}<br>Humidity: {row['Humidity']}%"
    folium.CircleMarker(location=[row['Lat'], row['Lng']], radius=row['Humidity'] / 5, popup=popup_text, fill=True, color='blue', fill_opacity=0.6).add_to(mymap)

# Display the map
mymap.save('humidity_map.html')


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:

# Assuming city_data_df is your DataFrame with city data

# Criteria for narrowing down cities
temperature_threshold = 25 
humidity_threshold = 50  

# Narrow down cities based on criteria
filtered_cities = city_data_df[
    (city_data_df['Temperature'] > temperature_threshold) &
    (city_data_df['Humidity'] > humidity_threshold)
]

# Drop any rows with null values
filtered_cities.dropna(inplace=True)

# Display sample data of the filtered cities
print(filtered_cities.head())



KeyError: 'Temperature'

In [40]:


# Assuming city_data_df is your DataFrame with city data

# Criteria for narrowing down cities
temperature_threshold = 25
humidity_threshold = 50

# Narrow down cities based on criteria
filtered_cities = city_data_df[
    (city_data_df['Temperature'] > temperature_threshold) &
    (city_data_df['Humidity'] > humidity_threshold)
]

# Drop any rows with null values
filtered_cities.dropna(inplace=True)

# Display sample data of the filtered cities
print(filtered_cities.head())


KeyError: 'Temperature'

### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
import pandas as pd

# Assuming city_data_df is your DataFrame with city data

# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = city_data_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())


                          City Country      Lat       Lng  Humidity Hotel Name
0                     tyukhtet      RU  56.5389   89.3011        97           
1  edinburgh of the seven seas      SH -37.0676  -12.3116        88           
2                      udachny      RU  66.4167  112.4000        25           
3                         okap      HT  19.7578  -72.2042        75           
4                     waitangi      NZ -43.9535 -176.5597        96           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
import pandas as pd
import requests

# Assuming hotel_df is your DataFrame with city data, including columns 'City', 'Country', 'Lat', 'Lng', 'Humidity', and 'Hotel Name'

# Set parameters to search for a hotel
radius = 5000  # Define the radius (in meters) within which to search for a hotel
params = {
    "apiKey": "YOUR_GEOAPIFY_API_KEY",  # Replace with your Geoapify API key
    "type": "hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df.head())


Starting hotel search
tyukhtet - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
okap - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
tofol - nearest hotel: No hotel found
ceeldheer - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
ooty - nearest hotel: No hotel found
una - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
altamira - nearest hotel: No hotel found
sakakah - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
afaahiti - nearest 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [31]:

%%capture --no-display
# Assuming hotel_df is your DataFrame with city data, including columns 'City', 'Country', 'Lat', 'Lng', 'Humidity', and 'Hotel Name'

# Create a map centered at a specific location
map_center = [hotel_df['Lat'].iloc[0], hotel_df['Lng'].iloc[0]]
map_zoom = 5  # Adjust the zoom level as needed

# Create the map
mymap = folium.Map(location=map_center, zoom_start=map_zoom)

# Add a point for each city with the size representing the humidity
for index, row in hotel_df.iterrows():
    popup_text = f"City: {row['City']}<br>Country: {row['Country']}<br>Humidity: {row['Humidity']}%<br>Hotel: {row['Hotel Name']}"
    folium.CircleMarker(location=[row['Lat'], row['Lng']], radius=row['Humidity'] / 5, popup=popup_text, fill=True, color='blue', fill_opacity=0.6).add_to(mymap)

# Display the map
mymap.save('humidity_map_with_hotels.html')
